In [1]:
import gymnasium as gym
import rlenvs
import numpy as np
from rlenvs import DiscreteCartPole
from functools import reduce


# env = gym.make("CartPole-v1")
# env = gym.make("custom/CartPole-v1")
env = gym.make("custom/DiscreteCartPole-v1")

# observation, info = env.reset(seed=82)
# observation, info = env.reset(seed=82, options={"sizes":(1, 20, 20, 20)})
observation, info = env.reset(seed=82, options={'masspole':.45, 'length':1.0, "sizes":(1, 20, 20, 20)})

/home/gui/Área de Trabalho/rlenvs/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:167: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/gui/Área de Trabalho/rlenvs/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:180: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


In [4]:
print(env.unwrapped.state)
print(env.step(1))
print([round(float(v), 5) for v in  apply_step(env.unwrapped.state)])
print(env.unwrapped.discretizer.__locate_in_space__(3, apply_step(env.unwrapped.state)[3]))
# print(np.array(obj.discretize(my_step(env.unwrapped.state))[1], dtype=np.float32))

[ 0.       0.      -0.02102 -0.14798]
(array([ 0.     ,  0.     , -0.02102, -0.34092], dtype=float32), 1.0, False, False, {})
[0.0, 0.19541, -0.02784, -0.64016]
(1, -0.61267)


In [8]:
env.unwrapped.discretizer.factor_spaces[2]

[-0.09834,
 -0.07621,
 -0.05698,
 -0.0406,
 -0.02702,
 -0.01619,
 -0.00809,
 -0.0027,
 0,
 0.0027,
 0.00809,
 0.01619,
 0.02702,
 0.0406,
 0.05698,
 0.07621,
 0.09834]

In [3]:
def apply_step(state, action=1):    
    gravity = 9.8
    masscart = 1.0
    masspole = 0.1
    total_mass = masspole + masscart
    length = 0.5  # actually half the pole's length
    polemass_length = masspole * length
    force_mag = 10.0
    tau = 0.02  # seconds between state updates

    x, x_dot, theta, theta_dot = state
    force = force_mag if action == 1 else -force_mag
    costheta = np.cos(theta)
    sintheta = np.sin(theta)

    temp = (
        force + polemass_length * np.square(theta_dot) * sintheta
    ) / total_mass
    thetaacc = (gravity * sintheta - costheta * temp) / (
        length
        * (4.0 / 3.0 - masspole * np.square(costheta) / total_mass)
    )
    xacc = temp - polemass_length * thetaacc * costheta / total_mass

    x = x + tau * x_dot
    x_dot = x_dot + tau * xacc
    theta = theta + tau * theta_dot
    theta_dot = theta_dot + tau * thetaacc


    return np.array((x, x_dot, theta, theta_dot), dtype=np.float32)

In [104]:
def factor(f, l, variate=lambda x: x%2==0):
    s = [0,0,0,0]
    counter = 0
    a = 1
    v_dot = set()
    v = set()
    for counter in range(100):
        if s[f]<-l or s[f]>l:
            break
        v_dot.add(round(float(s[f+1]), 5))
        v.add(round(float(s[f]), 5))
        counter+=1
        a = int( variate(counter))
        s = apply_step(s, a)

    v_dot.update({i*-1 for i in v_dot})
    v.update({i*-1 for i in v})
    v_dot = sorted(v_dot)
    v = [-l] + sorted(v) + [l]
    return v, v_dot

theta, theta_dot = factor(f=2, l=.21)
x, x_dot = factor(f=0, l=2.4, variate=lambda x: [0, 0, 1][x%3])
# x, x_dot = factor(f=0, l=2.4, variate=lambda x: False)
print(len(theta), theta)
print(len(theta_dot), theta_dot)
print(len(x), x)
print(len(x_dot), x_dot)

63 [-0.21, -0.20413, -0.18617, -0.17505, -0.15916, -0.14998, -0.13587, -0.12836, -0.11577, -0.10968, -0.09839, -0.09351, -0.08333, -0.07948, -0.07023, -0.06725, -0.05879, -0.05654, -0.04875, -0.0471, -0.03985, -0.03869, -0.03189, -0.03113, -0.02468, -0.02423, -0.01804, -0.01782, -0.01182, -0.01174, -0.00585, 0.0, 0.00585, 0.01174, 0.01182, 0.01782, 0.01804, 0.02423, 0.02468, 0.03113, 0.03189, 0.03869, 0.03985, 0.0471, 0.04875, 0.05654, 0.05879, 0.06725, 0.07023, 0.07948, 0.08333, 0.09351, 0.09839, 0.10968, 0.11577, 0.12836, 0.13587, 0.14998, 0.15916, 0.17505, 0.18617, 0.20413, 0.21]
63 [-0.89827, -0.79463, -0.7057, -0.6694, -0.62956, -0.56453, -0.55595, -0.50918, -0.46227, -0.45869, -0.42274, -0.38969, -0.37546, -0.36238, -0.34021, -0.32267, -0.30938, -0.3044, -0.30006, -0.29452, -0.29268, -0.24388, -0.19252, -0.14915, -0.11276, -0.08253, -0.05775, -0.03787, -0.02244, -0.01113, -0.00369, 0.0, 0.00369, 0.01113, 0.02244, 0.03787, 0.05775, 0.08253, 0.11276, 0.14915, 0.19252, 0.24388, 0.29

In [109]:
def get_factor_limits(simplify=True):
        # end_conditions = {0:[2.4], 2: [0.21]}
        def factor(f, l, variate):
                s = [0,0,0,0]
                counter = 0
                a = 1
                v_dot = set()
                v = set()
                for counter in range(100):
                        if s[f]<-l or s[f]>l:
                                break
                        v_dot.add(round(float(s[f+1]), 5))
                        v.add(round(float(s[f]), 5))
                        counter+=1
                        a = int( variate(counter))
                        s = apply_step(s, a)

                v_dot.update({i*-1 for i in v_dot})
                v.update({i*-1 for i in v})
                v_dot = sorted(v_dot)
                v = [-l] + sorted(v) + [l]
                return v, v_dot
        theta, theta_dot = factor(f=2, l=.21, variate=lambda x: x%2==0)
        if not simplify:
                x, x_dot = factor(f=0, l=2.4, variate=lambda x: [0, 0, 1][x%3])
                return [x, x_dot, theta, theta_dot]
        else:
                return [[], [], theta, theta_dot]

get_factor_limits()

[[],
 [],
 [-0.21,
  -0.20413,
  -0.18617,
  -0.17505,
  -0.15916,
  -0.14998,
  -0.13587,
  -0.12836,
  -0.11577,
  -0.10968,
  -0.09839,
  -0.09351,
  -0.08333,
  -0.07948,
  -0.07023,
  -0.06725,
  -0.05879,
  -0.05654,
  -0.04875,
  -0.0471,
  -0.03985,
  -0.03869,
  -0.03189,
  -0.03113,
  -0.02468,
  -0.02423,
  -0.01804,
  -0.01782,
  -0.01182,
  -0.01174,
  -0.00585,
  0.0,
  0.00585,
  0.01174,
  0.01182,
  0.01782,
  0.01804,
  0.02423,
  0.02468,
  0.03113,
  0.03189,
  0.03869,
  0.03985,
  0.0471,
  0.04875,
  0.05654,
  0.05879,
  0.06725,
  0.07023,
  0.07948,
  0.08333,
  0.09351,
  0.09839,
  0.10968,
  0.11577,
  0.12836,
  0.13587,
  0.14998,
  0.15916,
  0.17505,
  0.18617,
  0.20413,
  0.21],
 [-0.89827,
  -0.79463,
  -0.7057,
  -0.6694,
  -0.62956,
  -0.56453,
  -0.55595,
  -0.50918,
  -0.46227,
  -0.45869,
  -0.42274,
  -0.38969,
  -0.37546,
  -0.36238,
  -0.34021,
  -0.32267,
  -0.30938,
  -0.3044,
  -0.30006,
  -0.29452,
  -0.29268,
  -0.24388,
  -0.19252,
  -0

In [19]:
import numpy as np
from rlenvs import DiscreteCartPole

obj = DiscreteCartPole()

_, values = obj.discretize(observation)
print(observation)
print(np.array(values, dtype=np.float32))
print(((observation-values)**2)**.5)
# env.state = np.array(values, dtype=np.float64)
# env.unwrapped.state = np.array(values, dtype=np.float64)


[-0.          0.07407407 -0.          0.05555556]
[-0.          0.16666667  0.105       0.15713485]
[0.         0.09259259 0.105      0.10157928]


In [35]:
observation,reward, terminated, truncated, info = env.step(1)
_, values = obj.discretize(observation)
print(observation, terminated)
print(np.array(values, dtype=np.float32))
print(((observation-values)**2)**.5)
# env.state = np.array(values, dtype=np.float64)
# env.state = env.unwrapped.state = np.array(values, dtype=np.float64)

[-0.     1.5    0.105 -1.5  ] False
[-0.     1.5    0.105 -1.5  ]
[0.00000000e+00 2.22044605e-16 3.27825546e-09 2.22044605e-16]


In [36]:
print(env.unwrapped.state)
print(env.step(1)[0])
print(my_step(env.unwrapped.state))
print(np.array(obj.discretize(my_step(env.unwrapped.state))[1], dtype=np.float32))

[-0.     1.5    0.105 -1.5  ]
[-0.     1.5    0.105 -1.5  ]
[ 0.03       1.6937015  0.075     -1.7581388]
[-0.     1.5    0.105 -1.5  ]


In [37]:
obj.factor_spaces

[array([-0.]),
 array([-1.5       , -1.18518519, -0.90740741, -0.66666667, -0.46296296,
        -0.2962963 , -0.16666667, -0.07407407, -0.01851852, -0.        ,
         0.01851852,  0.07407407,  0.16666667,  0.2962963 ,  0.46296296,
         0.66666667,  0.90740741,  1.18518519,  1.5       ]),
 array([-0.21 , -0.105, -0.   ,  0.105,  0.21 ]),
 array([-1.5       , -1.25707872, -1.02890329, -0.81649658, -0.62112999,
        -0.44444444, -0.28867513, -0.15713484, -0.05555556, -0.        ,
         0.05555556,  0.15713484,  0.28867513,  0.44444444,  0.62112999,
         0.81649658,  1.02890329,  1.25707872,  1.5       ])]

In [55]:
[1,2,3,4,5][:0:-1]

[5, 4, 3, 2]

In [56]:
def teste(n):
    values = [[0, 0, 0, 0]]
    for i in range((n+1)//2):
        values.append(values[-1])
    return list(zip(*(values+values[:0:-1])))

teste(10)

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [44]:
my_step([0, 0, 0, 0], 1)

array([ 0.        ,  0.19512194,  0.        , -0.29268292], dtype=float32)

In [5]:
def my_step(state, action=1):    
    gravity = 9.8
    masscart = 1.0
    masspole = 0.1
    total_mass = masspole + masscart
    length = 0.5  # actually half the pole's length
    polemass_length = masspole * length
    force_mag = 10.0
    tau = 0.02  # seconds between state updates

    x, x_dot, theta, theta_dot = state
    force = force_mag if action == 1 else -force_mag
    costheta = np.cos(theta)
    sintheta = np.sin(theta)

    temp = (
        force + polemass_length * np.square(theta_dot) * sintheta
    ) / total_mass
    thetaacc = (gravity * sintheta - costheta * temp) / (
        length
        * (4.0 / 3.0 - masspole * np.square(costheta) / total_mass)
    )
    xacc = temp - polemass_length * thetaacc * costheta / total_mass

    x = x + tau * x_dot
    x_dot = x_dot + tau * xacc
    theta = theta + tau * theta_dot
    theta_dot = theta_dot + tau * thetaacc


    return np.array((x, x_dot, theta, theta_dot), dtype=np.float32)